# Xopt Evaluator Basic Usage 

In [ ]:
# needed for macos
import platform

if platform.system() == "Darwin":
    import multiprocessing as mp

    mp.set_start_method("fork")

In [ ]:
from xopt import Xopt, Evaluator, Generator, VOCS
from xopt.generators.random import RandomGenerator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt.evaluator import DummyExecutor
import numpy as np
import pandas as pd
from time import sleep
from random import randint

In [ ]:
vocs = VOCS(variables={"x": [0, 1], "y": [0, 1]}, objectives={"f1": "MINIMIZE"})


def f(inputs):

    sleep(randint(1, 5))  # simulate computation time
    # Make some occasional errors
    if inputs["x"] > 0.8:
        raise ValueError("x > 0.8")

    return {"f1": inputs["x"] ** 2 + inputs["y"] ** 2}


in1 = vocs.random_inputs()
f(in1)

In [ ]:
X = Xopt(generator=RandomGenerator(vocs), evaluator=Evaluator(f), vocs=vocs)

# Submit to the evaluator some new inputs
X.submit_data(vocs.random_inputs(4))

# Unevaluated inputs are collected in a dataframe
X._input_data

In [ ]:
# Internal futures dictionary
X._futures

In [ ]:
# Collect all finished futures and updata dataframe
X.update_data()
X.data

In [ ]:
# Futures are now cleared out
X._futures

In [ ]:
# This is the internal counter
X._ix_last

In [ ]:
X.submit_data(vocs.random_inputs(4))
X._input_data

In [ ]:
X._futures

In [ ]:
# Singe generation step
X.step()
X.update_data()
X.data

# Executors

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
MAX_WORKERS = 10

In [ ]:
# Dask
# from dask.distributed import Client
# import logging
# client = Client( silence_logs=logging.ERROR)
# executor = client.get_executor()
# client

In [ ]:
executor = "ThreadPoolExecutor"
# executor = ProcessPoolExecutor(max_workers=MAX_WORKERS)
executor

In [ ]:
X2 = Xopt(
    generator=RandomGenerator(vocs),
    evaluator=Evaluator(f, executor=executor, max_workers=MAX_WORKERS),
    vocs=vocs,
)
X2.options.asynch = True

In [ ]:
for _ in range(20):
    X2.step()

In [ ]:
X2.data.plot.scatter("x", "y")

In [ ]:
X2.data